# Q3: Even deeper! Resnet18 for PASCAL classification (15 pts)

Hopefully we all got much better accuracy with the deeper model! Since 2012, much deeper architectures have been proposed. [ResNet](https://arxiv.org/abs/1512.03385) is one of the popular ones. In this task, we attempt to further improve the performance with the “very deep” ResNet-18 architecture.


## 3.1 Build ResNet-18 (1 pts)
Write a network modules for the Resnet-18 architecture (refer to the original paper). You can use `torchvision.models` for this section, so it should be very easy! 
Do not load the pretrained weights for this question. We will get to that in the next question.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!tar -xf VOCtrainval_06-Nov-2007.tar

!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar 
!tar -xf VOCtest_06-Nov-2007.tar

--2022-03-01 00:32:38--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  19.3MB/s    in 22s     

2022-03-01 00:33:00 (19.6 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]

--2022-03-01 00:33:02--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (430M) [application/x-tar]
Saving to: ‘VOCtest_06-Nov-2007.tar’

VOCtest_06-Nov-2007 100%[===================

In [ ]:
%cd drive/MyDrive/spring22/16824/hw1

/content/drive/MyDrive/spring22/16824/hw1


In [ ]:
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import matplotlib.pyplot as plt
%matplotlib inline

import trainer
from utils import ARGS
from simple_cnn import SimpleCNN
from voc_dataset import VOCDataset

model = models.resnet18()
n_inputs = model.fc.in_features
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(n_inputs, 20))
]))
model.fc = classifier

## 3.2 Add Tensorboard Summaries (6 pts)
You should've already written tensorboard summary generation code into `trainer.py` from q1. However, you probably just added the most basic summary features. Please implement the more advanced summaries listed here:
* training loss (should be done)
* testing MAP curves (should be done)
* learning rate
* [histogram of gradients](https://www.tensorflow.org/api_docs/python/tf/summary/histogram)

## 3.3 Train and Test (8 pts)
Use the same hyperparameter settings from Task 2, and train the model for 50 epochs. Tune hyperparameters properly to get mAP around 0.5. Report tensorboard screenshots for *all* of the summaries listed above (for image summaries show screenshots at $n \geq 3$ iterations). For the histograms, include the screenshots of the gradients of layer1.1.conv1.weight and layer4.0.bn2.bias.

**REMEMBER TO SAVE A MODEL AT THE END OF TRAINING**

In [ ]:
args = ARGS(epochs=70,
            batch_size=32,
            lr=0.0005,
            use_cuda=True,
            step_size=30,
            save_freq=20,
            save_at_end=True,
            val_every=250)
model_name = 'ResNet9'
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)
test_ap, test_map = trainer.train(args, model, optimizer, scheduler, model_name=model_name)
print('test map:', test_map)

Train Epoch: 0 [0 (0%)]	Loss: 0.680367
Test MAP: 0.07267379250981906
Train Epoch: 0 [100 (64%)]	Loss: 0.228092
Train Epoch: 1 [200 (27%)]	Loss: 0.228333
Test MAP: 0.15880824333732985
Train Epoch: 1 [300 (91%)]	Loss: 0.218712
Train Epoch: 2 [400 (55%)]	Loss: 0.255169
Train Epoch: 3 [500 (18%)]	Loss: 0.221265
Test MAP: 0.1852035916981362
Train Epoch: 3 [600 (82%)]	Loss: 0.256896
Train Epoch: 4 [700 (46%)]	Loss: 0.191770
Test MAP: 0.22270256552115889
Train Epoch: 5 [800 (10%)]	Loss: 0.213907
Train Epoch: 5 [900 (73%)]	Loss: 0.217533
Train Epoch: 6 [1000 (37%)]	Loss: 0.218733
Test MAP: 0.16963835101949773
Train Epoch: 7 [1100 (1%)]	Loss: 0.205616
Train Epoch: 7 [1200 (64%)]	Loss: 0.211461
Test MAP: 0.27255797500866547
Train Epoch: 8 [1300 (28%)]	Loss: 0.203046
Train Epoch: 8 [1400 (92%)]	Loss: 0.228429
Train Epoch: 9 [1500 (55%)]	Loss: 0.193764
Test MAP: 0.2841128683142301
Train Epoch: 10 [1600 (19%)]	Loss: 0.203189
Train Epoch: 10 [1700 (83%)]	Loss: 0.194310
Test MAP: 0.3164940296288759
T

In [ ]:
%load_ext tensorboard

%tensorboard --logdir runs/ResNet3/